In [1]:
# Reference: https://www.kaggle.com/dansbecker/transfer-learning
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib
import os
import cv2
import random
import keras

from glob import glob
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.applications.vgg19 import VGG19
import tensorflow as tf
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
# loading the directories 
training_dir = 'C:/Users/avadh/Desktop/CMPE 295A/Dataset/Temp/Semi_Train'
test_dir = 'C:/Users/avadh/Desktop/CMPE 295A/Dataset/Temp/Semi_Test'
folders=os.listdir("C:/Users/avadh/Desktop/CMPE 295A/Dataset/Temp/Semi_Train")
print(folders)
#test_dir = '../input/fruits/fruits-360_dataset/fruits-360/test-multiple_fruits/'

['Chiffon', 'Jeans', 'Lace_Dress', "Men's_Shirts", 'Skirt']


In [4]:
size=64,64

In [5]:
# useful for getting number of files
image_files = glob(training_dir + '/*/*.jp*g')
test_image_files = glob(test_dir + '/*/*.jp*g')

In [6]:
image_train=[]
image_labels=[]
image_names=[]

for folder in folders:
    for each in os.listdir(os.path.join(training_dir,folder)):
        if each.endswith('jpg'):
            image_names.append(os.path.join(training_dir,folder,each))
            image_labels.append(folder)
            img=cv2.imread(os.path.join(training_dir,folder,each))
            img_in=cv2.resize(img,size)
            image_train.append(img_in)
        else:
            continue


In [7]:
union_list=list(zip(image_train,image_labels))
random.shuffle(union_list)
train,labels=zip(*union_list)
X=np.array(train)
Y=np.array(labels)

In [8]:
X=X/255.0


In [9]:
le=LabelEncoder()
Y=le.fit_transform(Y)
Y=to_categorical(Y,num_classes=len(folders))

In [10]:
X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=0.1)


In [11]:
vgg19_model=VGG19(include_top=False,input_shape=(64,64,3))

In [12]:
model=Sequential()

for layers in vgg19_model.layers:
    model.add(layers)
    
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))

for layer in vgg19_model.layers:
    layer.trainable=False

for layers in vgg19_model.layers[-2:]:
    layers.trainable=True

In [13]:
model.add(Dense(len(folders),activation='softmax'))


In [14]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 16, 16, 256)       295168    
__________

In [15]:
optimizer=RMSprop(lr=0.0001)   #,rho=0.9,epsilon=1e-08,decay=0.0)
model.compile(optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
learning_rate_reduction=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,minlr=0.00001)

In [17]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=False,
        vertical_flip=False) 

datagen.fit(X_train)

In [ ]:
history=model.fit_generator(datagen.flow(X_train,Y_train,batch_size=10),epochs=5,validation_data=(X_val,Y_val),
                          verbose=1,steps_per_epoch=X_train.shape[0]/10)


Epoch 1/5
536/535 [==============================] - 777s 1s/step - loss: 1.1685 - acc: 0.5368 - val_loss: 0.9767 - val_acc: 0.5872
Epoch 2/5
536/535 [==============================] - 628s 1s/step - loss: 0.9810 - acc: 0.6267 - val_loss: 0.8623 - val_acc: 0.6678
Epoch 3/5
536/535 [==============================] - 594s 1s/step - loss: 0.9098 - acc: 0.6556 - val_loss: 0.9167 - val_acc: 0.6577
Epoch 4/5
536/535 [==============================] - 587s 1s/step - loss: 0.8732 - acc: 0.6799 - val_loss: 0.8496 - val_acc: 0.7013
Epoch 5/5
449/535 [========================>.....] - ETA: 1:32 - loss: 0.8282 - acc: 0.6987

In [1]:
#Reference: https://machinelearningmastery.com/save-load-keras-deep-learning-models/

from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("Dataset/MyVersion/VGG16.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Using TensorFlow backend.


NameError: name 'model' is not defined

In [ ]:
# # getting the number of classes i.e. type of fruits
# folders = glob(training_dir + '/*')
# num_classes = len(folders)
# print ('Total Classes = ' + str(num_classes))

# size=64,64

In [ ]:
# # importing the libraries
# from keras.models import Model
# from keras.layers import Flatten, Dense
# from keras.applications import VGG16
# from keras.applications.vgg19 import VGG19
# #from keras.preprocessing import image

# IMAGE_SIZE = [64, 64]  # we will keep the image size as (64,64). You can increase the size for better results. 

# # loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
# vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# # this will exclude the initial layers from training phase as there are already been trained.
# for layer in vgg.layers:
#     layer.trainable = False

# x = Flatten()(vgg.output)
# #x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
# x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

# model = Model(inputs = vgg.input, outputs = x)

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# model.summary()


In [ ]:
# # Image Augmentation

# from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.vgg16 import preprocess_input

# training_datagen = ImageDataGenerator(
#                                     rescale=1./255,   # all pixel values will be between 0 an 1
#                                     shear_range=0.2, 
#                                     zoom_range=0.2,
#                                     horizontal_flip=True,
#                                     preprocessing_function=preprocess_input)

# # validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

# test_datagen = ImageDataGenerator(
#                                     rescale=1./255,   # all pixel values will be between 0 an 1
#                                     preprocessing_function=preprocess_input)


# training_generator = training_datagen.flow_from_directory(training_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')
# test_generator = test_datagen.flow_from_directory(test_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode= 'categorical')

# #validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')

In [ ]:
# # The labels are stored in class_indices in dictionary form. 
# # checking the labels
# training_generator.class_indices

In [ ]:
# #training_images = 15000
# #validation_images = 12709

# history = model.fit_generator(training_generator,
#                               steps_per_epoch = 201,  # this should be equal to total number of images in training set. But to speed up the execution, I am only using 10000 images. Change this for better results. 
#                               epochs = 3,
#                               validation_data = test_generator,
#                               validation_steps= 59)  # change this for better results
# #                    ,
# #                    validation_steps = 3000)  # this should be equal to total number of images in validation set.

In [ ]:
# model.evaluate()

In [ ]:
# print ('Training Accuracy = ' + str(history.history['acc']))
# #print ('Validation Accuracy = ' + str(history.history['val_acc']))

In [ ]:
# from keras.preprocessing import image;




In [ ]:
# temp = image.load_img('Dataset/img/Blouse/id_00000731/shop_01.jpg',target_size=(64,64));

In [ ]:
# temp = image.img_to_array(temp)
# temp = np.expand_dims(temp, axis=0)
# result = model.predict(temp)


In [ ]:
# result

In [ ]:
# training_generator.class_indices

In [ ]:
# #Reference: https://machinelearningmastery.com/save-load-keras-deep-learning-models/

# from keras.models import model_from_json

# # serialize model to JSON
# model_json = model.to_json()
# with open("Dataset/MyVersion/VGG16.json", "w") as json_file:
#     json_file.write(model_json)
    
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")

In [ ]:
# pwd
